# Test

- How to work with Googles API

In [1]:
import googlemaps
import os
from dotenv import load_dotenv
import pandas as pd

load_dotenv()

# Now you can access your API key like this

api_key = os.getenv('GOOGLE_MAPS_API_KEY')

# Initialize the Google Maps client with the API key

gmaps = googlemaps.Client(key=api_key)

Example London

In [2]:
location = '51.5074,-0.1278'  # Coordinates for London city center
radius = 10000  # 10 km radius

places_result = gmaps.places_nearby(location=location, radius=radius, type='park')

# Extract and store the results
parks = []
for place in places_result['results']:
    parks.append({
        'name': place['name'],
        'latitude': place['geometry']['location']['lat'],
        'longitude': place['geometry']['location']['lng'],
        'address': place.get('vicinity'),
    })

In [3]:
import time

# Initialize an empty list to hold all results
all_parks_data = []

# Define the cities and their parameters
cities = {
    "London": {"location": "51.5074,-0.1278", "radius": 20000},
    "Amsterdam": {"location": "52.3676,4.9041", "radius": 10000},
    "Milan": {"location": "45.4642,9.1900", "radius": 12000},
    "New York": {"location": "40.7128,-74.0060", "radius": 20000},
    "Portland": {"location": "45.5122,-122.6587", "radius": 15000},
    "Houston": {"location": "29.7604,-95.3698", "radius": 25000},
}

# Iterate through each city
for city, params in cities.items():
    places_result = gmaps.places_nearby(location=params["location"], radius=params["radius"], type='park')

    # Collect the first set of results
    parks = []
    for place in places_result['results']:
        parks.append({
            'city': city,
            'name': place['name'],
            'latitude': place['geometry']['location']['lat'],
            'longitude': place['geometry']['location']['lng'],
            'address': place.get('vicinity'),
        })

    # If there is a next_page_token, keep requesting more results
    while 'next_page_token' in places_result:
        time.sleep(2)  # Pause to avoid sending requests too quickly
        places_result = gmaps.places_nearby(location=params["location"], radius=params["radius"], type='park', page_token=places_result['next_page_token'])
        for place in places_result['results']:
            parks.append({
                'city': city,
                'name': place['name'],
                'latitude': place['geometry']['location']['lat'],
                'longitude': place['geometry']['location']['lng'],
                'address': place.get('vicinity'),
            })

    all_parks_data.extend(parks)

# Convert the list to a DataFrame
parks_df = pd.DataFrame(all_parks_data)
print(parks_df.head())

# Save the DataFrame to a CSV file
parks_df.to_csv('all_cities_parks.csv', index=False)

     city                             name   latitude  longitude  \
0  London                    Primrose Hill  51.539523  -0.160650   
1  London      K+K Hotel George Kensington  51.492352  -0.195519   
2  London  Flanders Fields Memorial Garden  51.500539  -0.135783   
3  London                      Brook Green  51.496182  -0.219833   
4  London                        Hyde Park  51.507364  -0.164113   

                        address  
0    Primrose Hill Road, London  
1  1-15 Templeton Place, London  
2           Westminster, London  
3           Brook Green, London  
4                        London  


In [6]:
import osmnx as ox

# Example for Hyde Park in London
park_name = "Hyde Park"
city_name = "London"

# Use osmnx to retrieve the park geometry
park = ox.geometries_from_place(f"{park_name}, {city_name}", tags={'leisure': 'park'})

# Reproject to a projected CRS (e.g., EPSG:3857)
park = park.to_crs(epsg=3857)

# Calculate the area in square meters
park['area_m2'] = park.geometry.area

# Display the area in square meters
print(park[['name', 'area_m2']])

                                        name       area_m2
element_type osmid                                        
way          3986346      Kensington Gardens  2.826859e+06
             230320621  Old Football Pitches  1.146619e+05
             372975520             Hyde Park  3.581643e+06


/var/folders/9x/21kgbkbn0gx6fdwpv8lnjmnh0000gn/T/ipykernel_25795/2373560056.py:8: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  park = ox.geometries_from_place(f"{park_name}, {city_name}", tags={'leisure': 'park'})


In [10]:
def get_park_area(park_name, city_name):
    try:
        # Try the specific park tag first
        park = ox.features_from_place(f"{park_name}, {city_name}", tags={'leisure': 'park'})
        
        # If not found, try broader queries
        if park.empty:
            park = ox.features_from_place(f"{park_name}, {city_name}", tags={'tourism': 'attraction'})
        if park.empty:
            park = ox.features_from_place(f"{park_name}, {city_name}", tags={'historic': True})
        if park.empty:
            park = ox.features_from_place(f"{park_name}, {city_name}")  # Broadest search with no tags

        # If still empty, skip this location
        if park.empty:
            return None

        # Reproject to a metric CRS and calculate the area
        park = park.to_crs(epsg=3857)
        return park.geometry.area.sum()
    
    except Exception as e:
        print(f"Could not find area for {park_name}: {e}")
        return None

# Apply the refined function to your dataset
for index, row in parks_df.iterrows():
    park_name = row['name']
    city_name = row['city']
    parks_df.at[index, 'area_m2'] = get_park_area(park_name, city_name)

Could not find area for K+K Hotel George Kensington: No data elements in server response. Check log and query location/tags.
Could not find area for Flanders Fields Memorial Garden: No data elements in server response. Check log and query location/tags.
Could not find area for WWT London Wetland Centre: Nominatim geocoder returned 0 results for query 'WWT London Wetland Centre, London'
Could not find area for The Phene: No data elements in server response. Check log and query location/tags.
Could not find area for Polish Air Force Memorial: Nominatim geocoder returned 0 results for query 'Polish Air Force Memorial, London'
Could not find area for Tower Hamlets Cemetery Park: No data elements in server response. Check log and query location/tags.
Could not find area for Lost Society: No data elements in server response. Check log and query location/tags.
Could not find area for Kenwood House: No data elements in server response. Check log and query location/tags.
Could not find area for

/opt/anaconda3/envs/env_test/lib/python3.11/site-packages/pyproj/transformer.py:820: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return self._transformer._transform_point(


Could not find area for Jackson Square: No data elements in server response. Check log and query location/tags.
Could not find area for Pier 63 at Hudson River Park: Nominatim geocoder returned 0 results for query 'Pier 63 at Hudson River Park, New York'
Could not find area for Brooklyn Bridge Park Pier 5: Nominatim could not geocode query 'Brooklyn Bridge Park Pier 5, New York' to a geometry of type (Multi)Polygon
Could not find area for Brooklyn War Memorial: No data elements in server response. Check log and query location/tags.
Could not find area for La Lanterna di Vittorio: Nominatim could not geocode query 'La Lanterna di Vittorio, New York' to a geometry of type (Multi)Polygon
Could not find area for Calvary Cemetery: No data elements in server response. Check log and query location/tags.


/opt/anaconda3/envs/env_test/lib/python3.11/site-packages/osmnx/features.py:1053: FutureWarning: <class 'geopandas.array.GeometryArray'>._reduce will require a `keepdims` parameter in the future
  gdf = gdf.dropna(axis="columns", how="all")


Could not find area for Juliette Restaurant: Nominatim could not geocode query 'Juliette Restaurant, New York' to a geometry of type (Multi)Polygon
Could not find area for One Brooklyn Bridge Park: No data elements in server response. Check log and query location/tags.
Could not find area for Pier 25 at Hudson River Park: No data elements in server response. Check log and query location/tags.
Could not find area for Aurora Brooklyn: Nominatim could not geocode query 'Aurora Brooklyn, New York' to a geometry of type (Multi)Polygon
Could not find area for Battery Park City Parks: Nominatim could not geocode query 'Battery Park City Parks, New York' to a geometry of type (Multi)Polygon
Could not find area for BOTTINO | modern italian dining | garden | events: Nominatim geocoder returned 0 results for query 'BOTTINO | modern italian dining | garden | events, New York'
Could not find area for Overpeck County Park: Nominatim geocoder returned 0 results for query 'Overpeck County Park, New Yo

/opt/anaconda3/envs/env_test/lib/python3.11/site-packages/osmnx/features.py:1053: FutureWarning: <class 'geopandas.array.GeometryArray'>._reduce will require a `keepdims` parameter in the future
  gdf = gdf.dropna(axis="columns", how="all")


Could not find area for Rocky Butte Natural Area: No data elements in server response. Check log and query location/tags.
Could not find area for Duniway Park Lilac Garden: Nominatim geocoder returned 0 results for query 'Duniway Park Lilac Garden, Portland'
Could not find area for Central Parking: Nominatim geocoder returned 0 results for query 'Central Parking, Portland'
Could not find area for Smith and Bybee Wetlands Natural Area: No data elements in server response. Check log and query location/tags.
Could not find area for Mill Ends Park - Worlds Smallest Park: Nominatim geocoder returned 0 results for query 'Mill Ends Park - Worlds Smallest Park, Portland'
Could not find area for Clinton City Park: Nominatim geocoder returned 0 results for query 'Clinton City Park, Portland'
Could not find area for Ladd Circle Park & Rose Gardens: Nominatim geocoder returned 0 results for query 'Ladd Circle Park & Rose Gardens, Portland'
Could not find area for Mount Talbert Nature Park: Nominat

In [11]:
parks_df['area_missing'] = parks_df['area_m2'].isnull()
missing_parks = parks_df[parks_df['area_missing']]

# Save the list of parks with missing area data for manual review
missing_parks.to_csv('missing_parks_for_review.csv', index=False)

In [12]:
dataset = pd.read_csv('missing_parks_for_review.csv')

In [13]:
dataset

,city,name,latitude,longitude,address,area_m2,area_missing
0,London,K+K Hotel George Kensington,51.492352,-0.195519,"1-15 Templeton Place, London",NaN,True
1,London,Flanders Fields Memorial Garden,51.500539,-0.135783,"Westminster, London",NaN,True
2,London,WWT London Wetland Centre,51.476141,-0.234665,"Queen Elizabeth Walk, London",NaN,True
3,London,The Phene,51.485174,-0.167575,"9 Phene Street, London",NaN,True
4,London,Polish Air Force Memorial,51.548843,-0.400308,"Western Avenue, Ruislip",NaN,True
...,...,...,...,...,...,...,...
200,Houston,Cane River Gardens,29.776295,-95.328099,"4205 Lyons Avenue, Houston",NaN,True
201,Houston,Paseo Park,29.705687,-95.464266,"5000 Bellaire Boulevard, Bellaire",NaN,True
202,Houston,Westbury Community Garden,29.659253,-95.477738,"5322 West Bellfort Avenue #100, Houston",NaN,True
203,Houston,"Houston Pools, Fountains, and Spas",29.804841,-95.421028,"1223 West 21st Street, Houston",NaN,True


In [17]:
dataset.area_m2.nunique()

0